In [ ]:
import jqfactor_analyzer as fa
import numpy as np 
import pandas as pd 
import random
from matplotlib import pyplot as plt
import factor_repo
import warnings
warnings.filterwarnings('ignore')

In [ ]:
a_date_bar_file_path = r'D:\QUANT_GAME\python_game\factor\factor_lab\get_date_bar\date_bar_post.pkl'
a_date_bar = pd.read_pickle(a_date_bar_file_path)
# a_date_bar = pd.read_pickle(r'D:\QUANT_GAME\python_game\pythonProject\DATA\local_stable_data\stock\CN_stock_data\dateBar.pkl')
idx = pd.IndexSlice

In [ ]:
def get_stk_list(date_bar):
    total_stk_list = list(date_bar.index.get_level_values(0).unique())
    index_codes_list = [idx_code  for idx_code in total_stk_list if (idx_code<'sh600000')|(idx_code>'sz399000')]
    stks_list = [security  for security in total_stk_list if security not in index_codes_list]
    return stks_list

def get_prepare_data(date_bar,fields,stk_list,del_paused=False,every_date_stk_nums=None,start_date=None,end_date=None)->pd.Series:
    data = date_bar.copy()
    data.index.names = ['stk','date']
    
    
    if stk_list:
        data = data.loc[idx[stk_list,:],:]
        
    if every_date_stk_nums:
        if del_paused:
            data = data[data['paused']==0]
    
        data = data.groupby('date',group_keys=False).apply(lambda s:s.sample(every_date_stk_nums) if len(s)>every_date_stk_nums else s)
        data.sort_index(inplace=True)
        
    if start_date:
        data = data.loc[idx[:,start_date:],:]
    if end_date:
        data = data.loc[idx[:,:end_date],:]
    if start_date and end_date:
        data = data.loc[idx[:,start_date:end_date],:]
    
    data = data.loc[:,fields]
    return data


In [ ]:
# 获取准备的日线
def get_prepare_date_bar(date_bar,start_date,end_date):
    stk_list = list(date_bar.index.get_level_values(0).unique())
    index_codes_list = [idx_code  for idx_code in stk_list if (idx_code<'sh600000')|(idx_code>'sz399000')]
    securities_list = [security  for security in stk_list if security not in index_codes_list]
    date_bar.index.names = ['stk','date']
    prepare_date_bar = date_bar.loc[idx[securities_list,:],:].groupby(\
        'stk',group_keys=False).apply(lambda df:df.ffill()).dropna()
    return prepare_date_bar.loc[idx[:,start_date:end_date],:]

In [ ]:
# # # 对factor用中位数去极值
# def winsorize_med(data, scale=2, inclusive=True, inf2nan=True, axis=0):
#    """
#    data: pd.Series/pd.DataFrame/np.array, 待缩尾的序列
#    scale: 倍数，默认为 1.0。会将位于 [med - scale * distance, med + scale * distance] 边界之外的值替换为边界值/np.nan
#    inclusive bool 是否将位于边界之外的值替换为边界值，默认为 True。 如果为 True，则将边界之外的值替换为边界值，否则则替换为 np.nan
#    inf2nan: 是否将 np.inf 和 -np.inf 替换成 np.nan，默认为 True。如果为 True，在缩尾之前会先将 np.inf 和 -np.inf 替换成 np.nan，缩尾的时候不会考虑 np.nan，否则 inf 被认为是在上界之上，-inf 被认为在下界之下
#    axis: 在 data 为 pd.DataFrame 时使用，沿哪个方向做标准化，默认为 1。0 为对每列做缩尾，1 为对每行做缩尾
#    """
#    if isinstance(data, pd.DataFrame):
#        if axis == 0:
#            data = data.apply(lambda x: _winsorize_med(x, scale, inclusive, inf2nan), axis=1)
#        elif axis == 1:
#            data = data.apply(lambda x: _winsorize_med(x, scale, inclusive, inf2nan), axis=0)
#        else:
#            raise ValueError("axis must be 0 or 1")
#    elif isinstance(data, pd.Series):
#        data = _winsorize_med(data, scale, inclusive, inf2nan)
#    elif isinstance(data, np.ndarray):
#        data = _winsorize_med(pd.Series(data), scale, inclusive, inf2nan).values
#    else:
#        raise ValueError("data must be pd.Series/pd.DataFrame/np.array")

#    return data

# def _winsorize_med(data, scale=1, inclusive=True, inf2nan=True):
#    if inf2nan:
#        data = pd.Series(data).replace([np.inf, -np.inf], np.nan)
#    else:
#        data = pd.Series(data)

#    data = data.astype(np.float64)
#    med = data.median()
#    distance = abs(data - med)
#    distance = distance.median()

#    lower_bound = med - scale * distance
#    upper_bound = med + scale * distance
#    if inclusive:
#        data[data < lower_bound] = lower_bound
#        data[data > upper_bound] = upper_bound
#    else:
#        data[data < lower_bound] = np.nan
#        data[data > upper_bound] = np.nan

#    return data


In [ ]:
# # 因子标准化
# def factor_standardization(factor_data,inf2nan=True,axis=1):
#     """
#     因子标准化
#     :param factor_data: 因子数据,dataframe
#     :param inf2nan: 是否将inf值替换为nan
#     :param axis: 标准化方向，0为列标准化，1为行标准化
#     :return: 标准化后的因子数据
#     """
#     factor_data = factor_data.copy()
#     if inf2nan:
#         factor_data[np.isinf(factor_data)] = np.nan
#     mean = np.nanmean(factor_data,axis=axis)
#     std = np.nanstd(factor_data,axis=axis)
#     factor_data = (factor_data.T - mean) / std
#     # print(len(mean))
#     return factor_data.T

In [ ]:
def get_prepare_data_of_analyze(prepare_date_bar,prepare_factor,winsorize_med_tf=False,scale=13,factor_standardization_tf=False):
    import datetime
    prepare_date_bar = prepare_date_bar.dropna()
    prepare_factor = prepare_factor.dropna()
    prepare_price = prepare_date_bar['close'].unstack().T
    del prepare_date_bar
    prepare_factor = prepare_factor.unstack().T 
    price_date_idx_list = prepare_price.index
    factor_date_idx_list = prepare_factor.index
    price_date_datetime_idx = [datetime.datetime.strptime(str(date_idx),'%Y%m%d') for date_idx in price_date_idx_list]
    factor_date_datetime_idx = [datetime.datetime.strptime(str(date_idx),'%Y%m%d') for date_idx in factor_date_idx_list]
    del price_date_idx_list,factor_date_idx_list
    prepare_price.index = price_date_datetime_idx
    prepare_factor.index = factor_date_datetime_idx
    del price_date_datetime_idx,factor_date_datetime_idx
    # if winsorize_med_tf:
    #     prepare_factor = winsorize_med(prepare_factor,scale,inclusive=True)
    # if factor_standardization_tf:
    #     prepare_factor = factor_standardization(prepare_factor,axis=1)
    return prepare_price,prepare_factor

In [ ]:
start_date,end_date = 20200101,20220101
check_start_date = start_date + 10000

In [ ]:
from scipy import stats
def calc_abs_ic(forward_returns:pd.Series,factor_data:pd.Series):
    returns = forward_returns.copy().dropna()
    factor_data = factor_data.copy().dropna()
    inner_index = factor_data.index.intersection(returns.index)
    returns = returns.loc[inner_index,]
    factor_data = factor_data.loc[inner_index,]
    ic = stats.pearsonr(returns,factor_data)
    return ic 

In [ ]:
prepare_date_bar = get_prepare_date_bar(a_date_bar,start_date,end_date)

In [ ]:
forward_return = prepare_date_bar['close'].groupby('stk',group_keys=False).pct_change(1).shift(-1)
periods = (1,)

## 1 price_speed_factor

In [ ]:
# 1_price_speed_factor
price_speed_factor = factor_repo.PriceSpeed(['close'],prepare_date_bar)
factor_data = price_speed_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)

abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods,max_loss=0.3)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')

## 2 price_speed_change_factor

In [ ]:
price_speed_change_factor = factor_repo.PriceSpeedChange(['close'],prepare_date_bar)
factor_data_a = price_speed_change_factor.calc()
# factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
periods = (1,)
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods,max_loss=0.3)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')


## 3.force_factor

In [ ]:
force_factor = factor_repo.Force(['close'],prepare_date_bar)
factor_data = force_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
# print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods,max_loss=0.3)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 4.impluse_factor

In [ ]:
impluse_factor = factor_repo.Impluse(['close'],prepare_date_bar)
factor_data = impluse_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
# print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
periods = (1,2,3,5,8,13,21,34,55)
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods,max_loss=0.3)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 5.max_min_price_average_factor

In [ ]:
max_min_price_average_factor = factor_repo.MaxMinPriceAverage(['close'],prepare_date_bar)
factor_data = max_min_price_average_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]

print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=3,periods=periods,max_loss=0.3)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 6.period_price_average_factor

In [ ]:
period_price_average_factor = factor_repo.PeriodPriceAverage(['close'],prepare_date_bar)
factor_data = period_price_average_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=3,periods=periods)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 7.Elevation_factor

In [ ]:
elevation_factor = factor_repo.Elevation(['close'],prepare_date_bar)
factor_data = elevation_factor.calc(zero_elevation_type='mean')
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)

In [ ]:
periods = (1,)
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 8.voltage factor

In [ ]:
voltage_factor = factor_repo.Voltage(['close'],prepare_date_bar)
factor_data = voltage_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)

In [ ]:
periods = (1,2,3,5,8,13,21,34,55)
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 9.current_factor

In [ ]:
current_factor = factor_repo.Current(['close'],prepare_date_bar)
# factor_data = current_factor.calc(zero_elevation_type='p(delta_t)')
factor_data = current_factor.calc(zero_elevation_type='mean')
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
abs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')

## 10.return_accumulation_factor

In [ ]:
return_accumulation_factor = factor_repo.ReturnAccumulation(['close'],prepare_date_bar)
factor_data = return_accumulation_factor.calc()
factor_data = factor_data.loc[idx[:,check_start_date:end_date]]
prepare_price,prepare_factor = get_prepare_data_of_analyze(prepare_date_bar,factor_data)
# prepare_factor = prepare_factor.loc[check_start_date:check_end_date,:]
print(prepare_factor.stack().describe())
prepare_factor.stack().hist(bins=200)
bs_ic = calc_abs_ic(forward_return,factor_data)
print('abs_ic:',abs_ic)

In [ ]:
factor_analyzer = fa.FactorAnalyzer(prices=prepare_price,factor=prepare_factor,quantiles=10,periods=periods)
# factor_analyzer.create_summary_tear_sheet()
# factor_analyzer.create_full_tear_sheet()
factor_analyzer.plot_information_table(method='normal')
factor_analyzer.plot_ic_hist(method='normal')
factor_analyzer.plot_ic_ts(method='normal')
# factor_analyzer.plot_ic_qq(method='normal')